# webscraping

In [2]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import re
import random
import time
import numpy as np

class RightmoveScraper:
    results = []  # Define a results list to store the scraped data

    # Method to send an HTTP GET request and fetch the HTML content of a URL
    def fetch(self, url):
        print('HTTP GET request to URL: %s' % url, end='')
        response = requests.get(url)  # Send an HTTP GET request
        print(' | Status code: %s' % response.status_code)  # Print the status code of the response

        return response

    # Method to parse the HTML content and extract property details
    def parse(self, html):
        content = BeautifulSoup(html, 'lxml')  # Create a BeautifulSoup object

        # Extract property titles, addresses, descriptions, prices, agents, bedrooms, and links using CSS selectors
        titles = [title.text.strip() for title in content.findAll('h2', {'class': 'propertyCard-title'})]
        addresses = [address['content'] for address in content.findAll('meta', {'itemprop': 'streetAddress'})]
        descriptions = [description.text for description in content.findAll('span', {'data-test': 'property-description'})]
        prices = [price.text.strip() for price in content.findAll('span', {'class': 'propertyCard-priceValue'})]
        if len(prices) == 0:
            prices = [price.text.strip() for price in content.findAll('div', {'class': 'propertyCard-priceValue'})]
        dates = [date.text.split(' ')[-1] for date in content.findAll('span', {'class': 'propertyCard-branchSummary-addedOrReduced'})]
        agent = [agent.text.split('by')[-1].strip() for agent in content.findAll('span', {'class': 'propertyCard-branchSummary-branchName'})]
        bedrooms = [title.text.split('bedroom')[0].strip() for title in content.findAll('h2', {'class': 'propertyCard-title'})]
        apartments = content.find_all("div", class_="l-searchResult is-list")
        agent_phone = [phone_number.text.strip() for phone_number in content.find_all('a', {'class': 'propertyCard-contactsPhoneNumber'}) ]

        
        all_apartment_links = []
        bathrooms = []
        for i in range(len(apartments)):
            apartment_no = apartments[i]  # Get the current apartment

            # Extract the link to the apartment
            apartment_info = apartment_no.find("a", class_="propertyCard-link")
            link = "https://www.rightmove.co.uk" + apartment_info.attrs["href"]
            all_apartment_links.append(link)  # Append the link to the list

        # Iterate over the property details and add them to the results list
        for index in range(0, len(titles)):
            self.results.append({
                'title': titles[index],
                'address': addresses[index],
                'description': descriptions[index],
                'listing_url': all_apartment_links[index],
                'price': prices[index],
                'agent': agent[index],
                'bedrooms': bedrooms[index]
            })

    # Method to convert the results list to a DataFrame
    def to_dataframe(self):
        return pd.DataFrame(self.results)

    # Method to transform and run the web scraping process
    def transform_run(self, url):
        response = self.fetch(url)  # Fetch the HTML content of the URL
        self.parse(response.text)  # Parse the HTML content and extract property details
        df = self.to_dataframe()  # Convert the results to a DataFrame
        df['listing_source'] = 'Rightmove'  # Add a column for the listing source

        if 'sale' in url:
            df['Transaction_type'] = 'sale'  
        else:
            df['Transaction_type'] = 'rent'  
        df['title'] = df['title'].str.replace('for sale', '')
        df['title'] = df['title'].str.replace('to rent', '')
        df['property_type'] = df['title'].apply(lambda x: re.search(r'(?<=bedroom\s).*', x, re.IGNORECASE).group() if re.search(r'(?<=bedroom\s).*', x, re.IGNORECASE) else None)
        df.property_type=df.property_type.str.strip()
        df['Postcode District'] = df['address'].str.extract(r'\b([A-Z]{1,2}\d{1,2}[A-Z]?)\b')

        df['price'] = df['price'].str.replace(',', '')
        df['price'] = df['price'].str.replace('£', '')
        df['price'] = df['price'].str.replace('pcm', '')
        df['bedrooms'] = df['bedrooms'].apply(lambda x: '0' if 'Studio' in x else x)
        df['bedrooms'] = df['bedrooms'].apply(lambda x: '' if len(x) > 2 else x)
        df['bedrooms'] = pd.to_numeric(df['bedrooms'], errors='coerce').astype('Int64')
       
        df.price = df.price.str.replace('POA', '')
        df['price'] = pd.to_numeric(df['price'], errors='coerce').astype('Int64')
        df = df.drop_duplicates(keep='first')  # Drop duplicate entries

        return df

if __name__ == '__main__':
    scraper = RightmoveScraper()

    # Scrape property data for sales
    for sales_page in range(0, 3):
        index = sales_page * 24
        if index == 0:
            sales_url = 'https://www.rightmove.co.uk/property-for-sale/find.html?locationIdentifier=REGION%5E87490&propertyTypes=&includeSSTC=false&mustHave=&dontShow=&furnishTypes=&keywords='
        elif index != 0:
            sales_url = f'https://www.rightmove.co.uk/property-for-sale/find.html?locationIdentifier=REGION%5E87490&index={index}&propertyTypes=&includeSSTC=false&mustHave=&dontShow=&furnishTypes=&keywords='
        sales_data = scraper.transform_run(sales_url)
        time.sleep(random.randint(1, 3))  # Pause for a random amount of time between requests

    # Scrape property data for rentals
    for rent_page in range(0, 3):
        index = rent_page * 24
        if index == 0:
            rent_url = 'https://www.rightmove.co.uk/property-to-rent/find.html?searchType=RENT&locationIdentifier=REGION%5E87490&insId=1&radius=0.0&minPrice=&maxPrice=&minBedrooms=&maxBedrooms=&displayPropertyType=&maxDaysSinceAdded=&sortByPriceDescending=&_includeLetAgreed=on&primaryDisplayPropertyType=&secondaryDisplayPropertyType=&oldDisplayPropertyType=&oldPrimaryDisplayPropertyType=&letType=&letFurnishType=&houseFlatShare='
        elif index != 0:
            rent_url = f'https://www.rightmove.co.uk/property-to-rent/find.html?locationIdentifier=REGION%5E87490&index={index}&propertyTypes=&includeLetAgreed=false&mustHave=&dontShow=&furnishTypes=&keywords='
        rent_data = scraper.transform_run(rent_url)
        time.sleep(random.randint(1, 3))  # Pause for a random amount of time between requests

    rightmove_data = pd.concat([sales_data, rent_data])  # Concatenate the sales and rentals data into a single DataFrame


HTTP GET request to URL: https://www.rightmove.co.uk/property-for-sale/find.html?locationIdentifier=REGION%5E87490&propertyTypes=&includeSSTC=false&mustHave=&dontShow=&furnishTypes=&keywords= | Status code: 200
HTTP GET request to URL: https://www.rightmove.co.uk/property-for-sale/find.html?locationIdentifier=REGION%5E87490&index=24&propertyTypes=&includeSSTC=false&mustHave=&dontShow=&furnishTypes=&keywords= | Status code: 200
HTTP GET request to URL: https://www.rightmove.co.uk/property-for-sale/find.html?locationIdentifier=REGION%5E87490&index=48&propertyTypes=&includeSSTC=false&mustHave=&dontShow=&furnishTypes=&keywords= | Status code: 200
HTTP GET request to URL: https://www.rightmove.co.uk/property-to-rent/find.html?searchType=RENT&locationIdentifier=REGION%5E87490&insId=1&radius=0.0&minPrice=&maxPrice=&minBedrooms=&maxBedrooms=&displayPropertyType=&maxDaysSinceAdded=&sortByPriceDescending=&_includeLetAgreed=on&primaryDisplayPropertyType=&secondaryDisplayPropertyType=&oldDisplayPr

In [6]:
from undetected_chromedriver import Chrome  # Import Chrome from undetected_chromedriver library
import time  # Import the time module
from bs4 import BeautifulSoup  # Import BeautifulSoup from bs4 library
import pandas as pd  # Import pandas library for data manipulation
import re  # Import re library for regular expressions
import random  # Import random library for generating random values

# Create a class named ZooplaScraper
class ZooplaScraper:
    results = []  # Define a results list to store the scraped data

    # Method to parse the HTML content and extract property details
    def parse(self, html):
        print('Scraping...')
        soup = BeautifulSoup(html, 'lxml')  # Create a BeautifulSoup object

        # Extract property prices, titles, addresses, descriptions, and agents using CSS selectors
        prices = [price.text.strip() for price in soup.find_all('p', {'data-testid': "listing-price"})]
        titles = [title.text.strip() for title in soup.find_all('h2', {'data-testid': "listing-title"})]
        addresses = [address.text.strip() for address in soup.findAll('h3', {'class': '_1ankud52 _1ftx2fq9'})]
        descriptions = [description.text.strip() for description in
                        soup.find_all('p', {'class': "_1ankud53 _1ftx2fq9"})]
        agents = [agent['alt'] for agent in soup.findAll('img', {'class': "_12bxhf70"})]

        links = soup.findAll('a', {'class': '_1maljyt1'})  # Find all property links
        all_apartment_links = []  # Create a list to store all property links

        # Extract the property links and append them to the all_apartment_links list
        for link in links:
            all_apartment_links.append('https://www.zoopla.co.uk' + link['href'])

        # Iterate over the property details and add them to the results list
        for index in range(0, len(titles)):
            self.results.append({
                'title': titles[index],
                'address': addresses[index],
                'description': descriptions[index],
                'listing_url': all_apartment_links[index],
                'price': prices[index],
                'agent': agents[index]
            })

    # Method to convert the results list to a DataFrame
    def to_dataframe(self):
        return pd.DataFrame(self.results)

    # Method to run the web scraping process
    def transform_run(self, url):
        chrome = Chrome()  # Create a new instance of Chrome
        chrome.get(url)  # Navigate to the specified URL

        html = chrome.page_source  # Get the HTML content of the page
        self.parse(html)  # Call the parse method to extract property details
        df = self.to_dataframe()  # Convert the results to a DataFrame

        # Perform additional data manipulation and feature extraction on the DataFrame
        df['bedrooms'] = df['title'].apply(lambda x: x.split()[0])
        df['bedrooms'] = df['bedrooms'].apply(lambda x: 0 if 'Studio' in x else x)
        df['bedrooms'] = pd.to_numeric(df['bedrooms'], errors='coerce').astype('Int64')
        df['listing_source'] = 'Zoopla'
        if 'sale' in url:
            df['Transaction_type'] = 'sale'
        else:
            df['Transaction_type'] = 'rent'
        df['title'] = df['title'].str.replace('for sale', '')
        df['title'] = df['title'].str.replace('to rent', '')
        df['property_type'] = df['title'].str.extract(r'bed\s+(.*)', flags=re.IGNORECASE)
        df['property_type'] = df['property_type'].str.strip()
        df['Postcode District'] = df['address'].str.extract(r'\b([A-Z]{1,2}\d{1,2}[A-Z]?)\b')
        df['price'] = df['price'].str.replace(',', '')
        df['price'] = df['price'].str.replace('£', '')
        df['price'] = df['price'].str.replace('pcm', '')
        df['price'] = df['price'].str.replace('POA', '')
        df['price'] = pd.to_numeric(df['price'], errors='coerce').astype('Int64')
        df = df.drop_duplicates(keep='first')
        chrome.quit()  # Close the browser instance
        return df
        
if __name__ == '__main__':
    scraper = ZooplaScraper()  # Create an instance of the ZooplaScraper class

    # Scrape property data for sales
    for index in range(0, 2):
        if index == 0:
            sales_url = 'https://www.zoopla.co.uk/for-sale/property/london/?q=London&search_source=home'
        elif index != 0:
            sales_url = f'https://www.zoopla.co.uk/for-sale/property/london/?q=London&search_source=home&pn={index}'
        zoopla_sales_data = scraper.transform_run(sales_url)
        time.sleep(random.randint(1, 5))  # Pause for a random amount of time between requests

    # Scrape property data for rentals
    for index in range(0, 2):
        if index == 0:
            rent_url = 'https://www.zoopla.co.uk/to-rent/property/london/?q=London&search_source=home'
        elif index != 0:
            rent_url = f'https://www.zoopla.co.uk/to-rent/property/london/?q=London&search_source=home&pn={index}'
        zoopla_rent_data = scraper.transform_run(rent_url)
        time.sleep(random.randint(1, 5))  

    zoopla_data = pd.concat([zoopla_sales_data, zoopla_rent_data])  # Concatenate the sales and rentals data


Scraping...
Scraping...
Scraping...
Scraping...


In [8]:
from undetected_chromedriver import Chrome  
import time  
from bs4 import BeautifulSoup  
import pandas as pd  
import re  
import random  

# Create a class named OTM
class OTM:
    results = []  # Define a results list to store the scraped data

    # Method to parse the HTML content and extract property details
    def parse(self, html):
        soup = BeautifulSoup(html, 'lxml')  # Create a BeautifulSoup object

        # Extract property titles, prices, and addresses using CSS selectors
        titles = [title.text for title in soup.findAll('span', {'class': 'title'})]
        prices = [price.text for price in soup.findAll('div', {'class': 'otm-Price'})]
        addresses = [address.text for address in soup.findAll('span', {'class': 'address'})]

        list_of_agents = soup.findAll('div', {'class': 'otm-PropertyCardAgent'})  # Find all agent details
        agents = []  # Create a list to store agent names

        # Extract agent names and append them to the agents list
        for i in list_of_agents:
            agent_name = i.find('small').text
            agent_name = re.sub('\s*Marketed by\s*', '', agent_name)
            agents.append(agent_name)

        links = soup.findAll('a', {'class': 'days-otm'})  # Find all property links
        all_apartment_links = []  # Create a list to store all property links

        # Extract the property links and append them to the all_apartment_links list
        for link in links:
            all_apartment_links.append('https://www.onthemarket.com' + link['href'])

        # Iterate over the property details and add them to the results list
        for index in range(0, len(titles)):
            self.results.append({
                'title': titles[index],
                'address': addresses[index],
                'listing_url': all_apartment_links[index],
                'price': prices[index],
                'agent': agents[index]
            })

    # Method to convert the results list to a DataFrame
    def to_dataframe(self):
        df = pd.DataFrame(self.results)  # Create a DataFrame from the results list
        return df

    # Method to run the web scraping process
    def transform_run(self, url):
        chrome = Chrome()  # Create a new instance of Chrome
        chrome.get(url)  # Navigate to the specified URL

        html = chrome.page_source  # Get the HTML content of the page
        self.parse(html)  # Call the parse method to extract property details
        df = self.to_dataframe()  # Convert the results to a DataFrame

        # Perform additional data manipulation and feature extraction on the DataFrame
        df['bedrooms'] = df.title.apply(lambda x: x.split()[0])
        df['bedrooms'] = df['bedrooms'].apply(lambda x: 0 if x == 'Studio' or x == 'Studio flat' else x)
        df['bedrooms']= pd.to_numeric(df['bedrooms'], errors='coerce').astype('Int64')
        df['listing_source'] = 'OTM'
        if 'sale' in url:
            df['Transaction_type'] = 'sale'
        else:
            df['Transaction_type'] = 'rent'
        df['title'] = df['title'].str.replace('for sale', '')
        df['title'] = df['title'].str.replace('to rent', '')
        df['property_type'] = df['title'].apply(
            lambda x: re.search(r'(?<=bedroom\s).*', x, re.IGNORECASE).group() if re.search(r'(?<=bedroom\s).*',
                                                                                            x, re.IGNORECASE) else None)
        df['property_type'] = df['property_type'].str.strip()
        df['Postcode District'] = df['address'].str.extract(r'\b([A-Z]{1,2}\d{1,2}[A-Z]?)\b')
        
        df['price'] = df['price'].str.replace('pcm', '')
        df['price'] = df['price'].apply(lambda x: re.search(r"£([\d,]+)", x).group(1).replace(",", "")
                                        if re.search(r"£([\d,]+)", x) else None)
        df['price'] = df['price'].str.replace('pcm', '')
        df['price']=pd.to_numeric(df['price'], errors='coerce').astype('Int64')
      
        df.insert(2, 'description', '')
        
        chrome.quit()  # Close the browser instance
        return df

        
if __name__ == '__main__':
    otm_scraper = OTM()  # Create an instance of the OTM class

    # Scrape property data for sales
    for index in range(0, 2):
        if index == 0:
            sales_url = 'https://www.onthemarket.com/for-sale/property/london/?view=grid'
        elif index != 0:
            sales_url = f'https://www.onthemarket.com/for-sale/property/london/?page={index}&view=grid'
        otm_sales_data = otm_scraper.transform_run(sales_url)
        time.sleep(random.randint(1, 3))  # Pause for a random amount of time between requests to avoid overwhelming the website

    # Scrape property data for rentals
    for index in range(0, 2):
        if index == 0:
            rent_url = 'https://www.onthemarket.com/to-rent/property/london/?view=grid'
        elif index != 0:
            rent_url = f'https://www.onthemarket.com/to-rent/property/london/?page={index}&view=grid'
        otm_rent_data = otm_scraper.transform_run(rent_url)
        time.sleep(random.randint(1, 3))  
    otm_data=pd.concat([otm_sales_data,otm_rent_data])

In [3]:
rightmove_data.head()

,title,address,description,listing_url,price,agent,bedrooms,listing_source,Transaction_type,property_type,Postcode District
0,1 bedroom apartment,"Union Wharf, Wenlock Road, N1",An outstanding and extremely well presented on...,https://www.rightmove.co.uk/properties/1350884...,850000,"Circa London, Shoreditch",1,Rightmove,sale,apartment,N1
1,5 bedroom apartment,"One Hyde Park, Knightsbridge",An exceptional exclusive five bedroom apartmen...,https://www.rightmove.co.uk/properties/1301776...,60000000,"The Cloister, London",5,Rightmove,sale,apartment,NaN
2,7 bedroom house,"Lygon Place, Belgravia, SW1W",Ref. LOB0798 - Set behind a gated Belgravia dr...,https://www.rightmove.co.uk/properties/1293029...,45000000,"Beauchamp Estates Ltd, Mayfair - Resale",7,Rightmove,sale,house,SW1W
3,12 bedroom house,"Mayfair Freehold House, Park Lane Area, W1K",This stunning 12 bedroom (including 3 staff be...,https://www.rightmove.co.uk/properties/1303065...,45000000,"Luxury Living Homes International, London",12,Rightmove,sale,house,W1K
4,7 bedroom detached house,"Pitt Street, London, W8","A rare opportunity to own this unique, contemp...",https://www.rightmove.co.uk/properties/1318464...,44000000,"Savills, Kensington",7,Rightmove,sale,detached house,W8


In [16]:
zoopla_data.head()

,title,address,description,listing_url,price,agent,bedrooms,listing_source,Transaction_type,property_type,Postcode District
0,2 bed flat,"Marlborough Road, Woolwich SE18","A spacious two bedroom, two bathroom apartment...",https://www.zoopla.co.uk/for-sale/details/6472...,495000,Benham and Reeves - Surrey Quays,2,Zoopla,sale,flat,SE18
1,2 bed semi-detached house,"Beauchamp Road, Sutton, Surrey SM1",This attractive Victorian semi-detached home i...,https://www.zoopla.co.uk/for-sale/details/6472...,475000,Burn & Warne,2,Zoopla,sale,semi-detached house,SM1
2,4 bed flat,"Iverna Gardens, London W8","A four bedroom, first floor lateral mansion fl...",https://www.zoopla.co.uk/for-sale/details/6472...,3100000,Savills - Kensington,4,Zoopla,sale,flat,W8
3,1 bed flat,"Queen's Gate, London SW7",A bright and spacious one bedroom apartment lo...,https://www.zoopla.co.uk/for-sale/details/6472...,900000,Savills - Knightsbridge,1,Zoopla,sale,flat,SW7
4,3 bed flat,"Lancaster Grove, London NW3",An impressive three bedroom ground floor flat ...,https://www.zoopla.co.uk/for-sale/details/6472...,2300000,Savills - Hampstead,3,Zoopla,sale,flat,NW3


In [17]:
otm_data.head()

,title,address,description,listing_url,price,agent,bedrooms,listing_source,Transaction_type,property_type,Postcode District
0,1 bedroom apartment,"Oswald building, London SW11",,https://www.onthemarket.com/details/12779184/,575000,JLL - Nine Elms,1,OTM,sale,apartment,SW11
1,2 bedroom flat,"Stephendale Road, Fulham, London, SW6",,https://www.onthemarket.com/details/13262722/,800000,Aspire - Fulham South,2,OTM,sale,flat,SW6
2,2 bedroom flat,"Broadfield Road, Catford",,https://www.onthemarket.com/details/13262059/,400000,Kinleigh Folkard & Hayward - Catford Sales,2,OTM,sale,flat,NaN
3,2 bedroom flat,"Millbank, London SW1P",,https://www.onthemarket.com/details/13261993/,2600000,Network Homes - Arc,2,OTM,sale,flat,SW1P
4,2 bedroom flat,"The Downs, Wimbledon",,https://www.onthemarket.com/details/13261935/,575000,Trevlyn Properties - Maidstone,2,OTM,sale,flat,NaN


In [33]:
data=pd.concat([rightmove_data,zoopla_data,otm_data])
# code to drop duplicates records using address, price, agent, bedrooms, property_type, transaction_type and post code distict column
data=data.drop_duplicates(subset=['address','agent','bedrooms','property_type','Transaction_type','Postcode District'],keep='first')

data

,title,address,description,listing_url,price,agent,bedrooms,listing_source,Transaction_type,property_type,Postcode District
0,2 bedroom apartment,"Madoc Close, Childs HIll","An immaculately presented two double bedroom, ...",https://www.rightmove.co.uk/properties/1323753...,725000,"Goldschmidt & Howland, West Hampstead - Sales",2,Rightmove,sale,apartment,NaN
1,5 bedroom apartment,"One Hyde Park, Knightsbridge",An exceptional exclusive five bedroom apartmen...,https://www.rightmove.co.uk/properties/1301776...,60000000,"The Cloister, London",5,Rightmove,sale,apartment,NaN
2,12 bedroom house,"Mayfair Freehold House, Park Lane Area, W1K",This stunning 12 bedroom (including 3 staff be...,https://www.rightmove.co.uk/properties/1303065...,45000000,"Luxury Living Homes International, London",12,Rightmove,sale,house,W1K
3,7 bedroom house,"Lygon Place, Belgravia, SW1W",Ref. LOB0798 - Set behind a gated Belgravia dr...,https://www.rightmove.co.uk/properties/1293029...,45000000,"Beauchamp Estates Ltd, Mayfair - Resale",7,Rightmove,sale,house,SW1W
4,7 bedroom detached house,"Pitt Street, London, W8","A rare opportunity to own this unique, contemp...",https://www.rightmove.co.uk/properties/1318464...,44000000,"Savills, Kensington",7,Rightmove,sale,detached house,W8
...,...,...,...,...,...,...,...,...,...,...,...
83,2 bedroom apartment,Campden Hill Gardens London W8,,https://www.onthemarket.com/details/13262967/,4311,Kinleigh Folkard & Hayward - Holland Park Lett...,2,OTM,rent,apartment,W8
84,1 bedroom flat,"Kingston Road, Wimbledon Chase, London, SW20 8BU",,https://www.onthemarket.com/details/6816802/,1850,Tennison Property - Wimbledon,1,OTM,rent,flat,SW20
85,1 bedroom flat,"Kara Court, Caspian Wharf, 15 Seven Sea Garden...",,https://www.onthemarket.com/details/13262942/,1690,Abby Properties - Isle of Dogs,1,OTM,rent,flat,E3
86,2 bedroom flat,"One Blackfrairs, 1-16 Blackfriars Road, London...",,https://www.onthemarket.com/details/13262941/,6716,Abby Properties - Isle of Dogs,2,OTM,rent,flat,SE1


In [34]:
data.to_csv('London_property_data.csv',index=False)